In [1]:
import numpy as np
from scipy.linalg import expm
from matplotlib import pyplot as plt
import cirq

In [2]:
ident = np.array([[1., 0.], [0., 1.]])
sigma_x = np.array([[0., 1.], [1., 0.]])
sigma_y = np.array([[0., -1.0j], [1.0j, 0.]])
sigma_z = np.array([[1., 0.], [0., -1.]])

In [3]:
def omega():
    return np.random.rand() * np.pi * 2
def phi ():
    return np.random.rand() * np.pi * 2
def theta():
    return np.arccos(2 * np.random.rand() - 1)

def unitary_angles(n_qubits):
    omegas = [omega() for i in range(n_qubits)]
    phis = [phi() for i in range(n_qubits)]
    thetas = [theta() for i in range(n_qubits)]

    return [phis,omegas,thetas]

def unitary(phi,omega,theta):
    
    return expm(-0.5j * omega * sigma_z) @ expm(-0.5j * theta * sigma_x) @ expm(-0.5j * phi * sigma_z)

In [4]:
qubits = cirq.LineQubit.range(3)

def shadows(angles):

    shadows = cirq.Circuit()

# Preparing GHZ state

    shadows.append([cirq.H(qubits[0]),
                    cirq.CNOT(qubits[0],qubits[1]),
                    cirq.CNOT(qubits[1],qubits[2])
                    ])

# Unitaries

    shadows.append([cirq.rz(angles[0][0])(qubits[0]), cirq.rz(angles[0][1])(qubits[1]), cirq.rz(angles[0][2])(qubits[2]),

                    cirq.rx(angles[2][0])(qubits[0]), cirq.rx(angles[2][1])(qubits[1]), cirq.rx(angles[2][2])(qubits[2]),

                    cirq.rz(angles[1][0])(qubits[0]), cirq.rz(angles[1][1])(qubits[1]), cirq.rz(angles[1][2])(qubits[2])],
                    strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)

# Measuring

    shadows.append(cirq.measure(qubits[0],qubits[1],qubits[2],key='out'))

    return shadows

print(shadows(unitary_angles(3)))


0: ───H───@───────Rz(0.93π)───Rx(0.559π)───Rz(1.21π)───M('out')───
          │                                            │
1: ───────X───@───Rz(1.66π)───Rx(0.461π)───Rz(1.17π)───M──────────
              │                                        │
2: ───────────X───Rz(1.75π)───Rx(0.679π)───Rz(0.35π)───M──────────


In [5]:
projectors = np.zeros([2,2,2])
projectors[0,0,0] = 1
projectors[1,1,1] = 1

def snapshot(angles,results):
    
    rho = np.eye(1)
    for i_mmt, mmt in enumerate(results):
        U = unitary(angles[0][i_mmt],angles[1][i_mmt],angles[2][i_mmt])
        rho = np.kron(rho, 3 * np.conj((U.T)) @ projectors[:,:,mmt] @ U - np.eye(2))
    
    return rho

N_rep = 10000
rho = np.zeros([8,8],dtype='complex')
for repetitions in range(N_rep):
    angles = unitary_angles(3)
    circuit = shadows(angles)
    simulator = cirq.Simulator()
    result = simulator.run(circuit).measurements['out'][0]

    rho += snapshot(angles,result)

rho /= N_rep

rho_ex = np.zeros([8,8])
rho_ex[0,0] = 1/2
rho_ex[7,7] = 1/2
rho_ex[0,7] = 1/2
rho_ex[7,0] = 1/2
print(np.trace(np.conj((rho-rho_ex).T)@(rho - rho_ex)))







(0.00863698193026017+0j)
